# Estimating Stop Frequency

This notebook illustrates how to re-estimate a single model component for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [ ]:
import os
import larch  # !conda install larch -c conda-forge # for estimation
import pandas as pd

In [ ]:
# update path to stop frequency estimation data bunldes!
path_to_stop_freq_EDBs = r'C:\ABM3_dev\outputs\output\estimation_data_bundle\stop_frequency'

for purpose in ['school', 'work', 'escort']:
    # ---- first creating backup of chooser file
    # creating paths for actual and backup
    orig_df_path = os.path.join(path_to_stop_freq_EDBs, purpose, 'stop_frequency_values_combined_orig.csv')
    actual_df_path = os.path.join(path_to_stop_freq_EDBs, purpose, 'stop_frequency_values_combined.csv')

    # if backup does not exist, create it. Otherwise, just read backup
    if ~os.path.exists(orig_df_path):
        df = pd.read_csv(actual_df_path)
        df.to_csv(orig_df_path, index=False)
    else:
        df = pd.read_csv(orig_df_path)

    # ---- updating stop frequency override_choice to remove trips on school escort tours
    def remove_stops_if_school_escorting(row):
        choices = row['override_choice'].split('_')
        out_choice = choices[0]
        in_choice = choices[1]
        if row['school_esc_outbound'] in ['ride_share', 'pure_escort']:
            out_choice = '0out'
        if row['school_esc_inbound'] in ['ride_share', 'pure_escort']:
            in_choice = '0in'
        return out_choice + '_' + in_choice


    # setting choice to 0 stops if half-tour includes school escorting
    df['override_choice'] = df.apply(lambda row: remove_stops_if_school_escorting(row), axis=1)

    # saving file to it is read in by rest of notebook
    df.to_csv(actual_df_path, index=False)



In [ ]:
pd.crosstab(df.override_choice, df.school_esc_inbound, margins=True)

In [ ]:
# writing code to create coefficients automatically based on what is in the stop_frequency files
tour_purposes = ['atwork', 'work', 'univ', 'school', 'eatout', 'social', 'othdiscr', 'othmaint', 'escort', 'shopping']
file_location = r'C:\ABM3_dev\ABM\src\asim\configs\estimation'
overwrite_all = True

for purpose in tour_purposes:
    spec_path = os.path.join(file_location, f'stop_frequency_{purpose}.csv')
    coefs_path = os.path.join(file_location, f'stop_frequency_coefficients_{purpose}.csv')

    spec = pd.read_csv(spec_path)
    coefs = pd.DataFrame(columns=['value', 'constrain', 'coefficient_name']).set_index('coefficient_name')
    coefs.loc['coef_unavailable'] = (-999, 'T')
    
    for index, row in spec.iterrows():
        util_name = row['Label']
        if util_name.startswith('#'):
            continue
        
        for col in spec.columns[4:]:
            old_coef_name = row[col]
            if old_coef_name == 'coef_unavailable':
                continue
            if ((old_coef_name is None) | overwrite_all):
                coef_name = util_name.replace('util', 'coef')
                coef_name = f'{coef_name}_{col}_{purpose}' 
            else:
                coef_name = old_coef_name
            spec.loc[index, col] = coef_name
            coefs.loc[coef_name] = (0.0, 'F')

    spec.to_csv(spec_path, index=False)
    coefs.to_csv(coefs_path, index=True)

# display(spec)
# display(coefs)


We'll work in our `test` directory, where ActivitySim has saved the estimation data bundles.

In [ ]:
os.chdir('C:\ABM3_dev\outputs')

# Load data and prep model for estimation

In [ ]:
modelname = "stop_frequency"

from activitysim.estimation.larch import component_model
model, data = component_model(modelname, return_data=True)

# Review data loaded from the EDB

The next step is to read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

In [ ]:
spec_segments = [i.primary_purpose for i in data.settings.SPEC_SEGMENTS]
spec_segments

## Coefficients

There is one meta-coefficients dataframe for this component, which contains
parameters for all the matching coefficients in the various segmented 
files. When different segments have the same named coefficient with the same
value, it is assumed they should be estimated jointly.  If they have the same name
but different values in the coefficient files, then they are re-estimated
independently.

In [ ]:
data.coefficients

## Utility specification

The utility spec files are unique to each segment model.  The estimation mode larch pre-processor
for the stop frequency model modifies the spec files to account for jointly re-estimated
parameters.

In [ ]:
# data.spec[0]

## Chooser data

The chooser data is unique to each segment model. 

In [ ]:
# data.chooser_data[0]

# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [ ]:
data.chooser_data[0].primary_purpose.value_counts()

In [ ]:
for i in range(0, len(data.chooser_data)):
    print(f"model {i} is purpose {data.chooser_data[i].primary_purpose.value_counts().idxmax()}")

In [ ]:
# model[0].estimate(method='BHHH') # can also estimate models one at a time
model.estimate(method='SLSQP', options={"maxiter": 1000})

### Estimated coefficients

In [ ]:
# model.parameter_summary()

# Output Estimation Results

The stop frequency model include seperate coefficient file for each segment,
and has a special writer method to seperate the coefficient by segment
after estimation.

In [ ]:
from activitysim.estimation.larch.stop_frequency import update_segment_coefficients
result_dir = data.edb_directory/"estimated"
update_segment_coefficients(
    model, data, result_dir,
    output_file="stop_frequency_coefficients_{segment_name}_revised.csv",
);

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [ ]:
for m, segment in zip(model, data.segments):
    m.to_xlsx(
        result_dir/f"{modelname}_{segment}_model_estimation.xlsx", 
        data_statistics=False,
    )

# Next Steps

The final step is to either manually or automatically copy the `stop_frequency_coefficients_*_revised.csv` files to the configs folder, rename them to `stop_frequency_coefficients_*.csv`, and run ActivitySim in simulation mode.

In [ ]:
pd.read_csv(result_dir/"stop_frequency_coefficients_work_revised.csv")